## 0. Imports and Constants
- Select user before proceeding
- Select feature methods before proceeding
- Select dataset version (default: v2)

In [1]:
############## AUTORELOAD MAGIC ###################
%load_ext autoreload
%autoreload 2
###################################################

############## FUNDAMENTAL MODULES ################
import json
from PIL import Image
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import pickle
import re
 ##################################################

############## TASK-SPECIFIC MODULES ##############
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from ml.datasets import VanillaDataset
from ml.models import VanillaNN
from ml.trainer import Trainer
###################################################


####################### CONSTANTS ########################
SPLITS = ["train", "dev", "test", "gold"]
TRAIN, DEV, TEST, TXT, IMG = "train", "dev", "test", "txt", "img"
#FE_METHODS = ["txt_embeddings", "img_embeddings", "concat", "sum", "mean", "hadamard"]
#FE_METHODS += ["concat_cos", "sum_cos", "mean_cos", "hadamard_cos"]
FE_METHODS = ["paraphrase", "paraphrase_with_ocr"]
##########################################################

############## DATA SCIENCE & ML MODULES #################
from transformers import CLIPTokenizerFast, CLIPProcessor, CLIPModel
import torch
import pandas as pd
from scipy import stats
from sklearn.svm import SVC
from sklearn.metrics import classification_report
##########################################################

####################### SELECT ###########################
users = ["patriziopalmisano", "onurdenizguler", "jockl"]
user = users[2] # SELECT USER
version = "v2" # SELECT DATASET VERSION
dataset_version = version
##########################################################

if user in users[:2]:
    data_dir = f"/Users/{user}/Library/CloudStorage/GoogleDrive-check.worthiness@gmail.com/My Drive/data/CT23_1A_checkworthy_multimodal_english_{version}"
    cw_dir = f"/Users/{user}/Library/CloudStorage/GoogleDrive-check.worthiness@gmail.com/My Drive"

else:
    data_dir = f"/home/jockl/Insync/check.worthiness@gmail.com/Google Drive/data/CT23_1A_checkworthy_multimodal_english_{dataset_version}"
    cw_dir = "/home/jockl/Insync/check.worthiness@gmail.com/Google Drive"

features_dir = f"{data_dir}/features"
labels_dir = f"{data_dir}/labels"
models_dir = f"{cw_dir}/models/vanillann_hyperparam_search"

# 1. Load the Datasets

In [2]:
split_to_labels = {split: 
                   np.load(f"{labels_dir}/{split}_labels_{dataset_version}.pickle", allow_pickle=True) if split != "gold" else np.load(f"{labels_dir}/{split}_labels.pickle", allow_pickle=True)
                   for split in SPLITS}

method_to_split_to_data = {fe_method: {
                                    split: 
                                    np.load(f"{features_dir}/{fe_method}/{fe_method}_{split}_{dataset_version}.pickle", allow_pickle=True) if split != "gold" else np.load(f"{features_dir}/{fe_method}/{fe_method}_{split}.pickle", allow_pickle=True)
                                    for split in SPLITS
                                    } 
                            for fe_method in FE_METHODS}

method_to_split_to_dataset = {fe_method: {
                                        split:
                                        VanillaDataset(method_to_split_to_data[fe_method][split], split_to_labels[split]) 
                                        for split in SPLITS
                                        } 
                                for fe_method in FE_METHODS}

# 2. Hyperparam Search

In this section, the same hyperparameter search is performed on different feature engineering methods. Every search features the following parameters:

In [3]:
# Hyperparams to search from
model_init_param_configs = [
    [128, 64, 32, 16],
    [128, 64, 32],
    [64, 32, 128],
    [128, 64],
    [64, 32],
    [32, 16],
    [64],
    [32],
    [16]
]
learning_rates = [1e-6, 5e-5, 1e-5, 5e-4, 1e-4]
batch_sizes = [8, 16, 32, 64, 128]
num_epochs = 1000

In [4]:
# Select device
device = "cpu"

## 2.1 Hyperparam Search on Text Embeddings

In [ ]:
# Select method 
dataset_method = "txt_embeddings"
dataset_dict = method_to_split_to_dataset[dataset_method]

# Select model architecture and model class
model = VanillaNN

# Instantiate a trainer on the selected model class, model architecture, and dataset
trainer_text = Trainer(model, dataset_method, dataset_dict)

In [ ]:
# Perform hyper param search on the selected method
for model_config in model_init_param_configs:
    trainer_text.hyperparameter_search(model_init_params = model_config,
                                    device = device,
                                    learning_rates = learning_rates,
                                    batch_sizes = batch_sizes,
                                    num_epochs = num_epochs)

In [ ]:
# Save all trained models
dir = f"{models_dir}/{dataset_method}"
trainer_text.save_trained_models(dir=dir, to_be_saved_models=None)

## 2.2 Hyperparam Search on Concat Embeddings

In [ ]:
# Select method 
dataset_method = "concat"
dataset_dict = method_to_split_to_dataset[dataset_method]

# Select model architecture and model class
model = VanillaNN

# Instantiate a trainer on the selected model class, model architecture, and dataset
trainer_concat = Trainer(model, dataset_method, dataset_dict)

In [ ]:
# Perform hyper param search on the selected method
for model_config in model_init_param_configs:
    trainer_concat.hyperparameter_search(model_init_params = model_config,
                                    device = device,
                                    learning_rates = learning_rates,
                                    batch_sizes = batch_sizes,
                                    num_epochs = num_epochs)

In [ ]:
# Save all trained models
dir = f"{models_dir}/{dataset_method}"
trainer_concat.save_trained_models(dir=dir, to_be_saved_models=None)

## 2.3 Hyperparam Search on Mean Embeddings

In [ ]:
# Select method 
dataset_method = "mean"
dataset_dict = method_to_split_to_dataset[dataset_method]

# Select model architecture and model class
model = VanillaNN

# Instantiate a trainer on the selected model class, model architecture, and dataset
trainer_mean = Trainer(model, dataset_method, dataset_dict)

In [ ]:
# Perform hyper param search on the selected method
for model_config in model_init_param_configs:
    trainer_mean.hyperparameter_search(model_init_params = model_config,
                                    device = device,
                                    learning_rates = learning_rates,
                                    batch_sizes = batch_sizes,
                                    num_epochs = num_epochs)

In [ ]:
# Save all trained models
dir = f"{models_dir}/{dataset_method}"
trainer_mean.save_trained_models(dir=dir, to_be_saved_models=None)

## 2.4 Hyperparam Search on Paraphrase Embeddings

In [5]:
# Select method 
dataset_method = "paraphrase"
dataset_dict = method_to_split_to_dataset[dataset_method]

# Select model architecture and model class
model = VanillaNN

# Instantiate a trainer on the selected model class, model architecture, and dataset
trainer = Trainer(model, dataset_method, dataset_dict)

In [6]:
# Perform hyper param search on the selected method
for model_config in model_init_param_configs:
    trainer.hyperparameter_search(model_init_params = model_config,
                                    device = device,
                                    learning_rates = learning_rates,
                                    batch_sizes = batch_sizes,
                                    num_epochs = num_epochs)

  0%|          | 0/1000 [00:00<?, ?it/s]

Early quitting at epoch: 396


  0%|          | 0/1000 [00:00<?, ?it/s]

Early quitting at epoch: 552


  0%|          | 0/1000 [00:00<?, ?it/s]

...

Early quitting at epoch: 22


  0%|          | 2/1000 [00:00<01:09, 14.40it/s]

Early quitting at epoch: 24


  2%|▏         | 24/1000 [00:01<01:12, 13.47it/s]

Early quitting at epoch: 24


In [7]:
# Save all trained models
dir = f"{models_dir}/{dataset_method}"
trainer.save_trained_models(dir=dir, to_be_saved_models=None)

## 2.5 Hyperparam Search on Paraphrase with OCR Embeddings

In [8]:
# Select method 
dataset_method = "paraphrase_with_ocr"
dataset_dict = method_to_split_to_dataset[dataset_method]

# Select model architecture and model class
model = VanillaNN

# Instantiate a trainer on the selected model class, model architecture, and dataset
trainer = Trainer(model, dataset_method, dataset_dict)

In [9]:
# Perform hyper param search on the selected method
for model_config in model_init_param_configs:
    trainer.hyperparameter_search(model_init_params = model_config,
                                    device = device,
                                    learning_rates = learning_rates,
                                    batch_sizes = batch_sizes,
                                    num_epochs = num_epochs)

  0%|          | 0/1000 [00:00<?, ?it/s]

Early quitting at epoch: 342


  0%|          | 0/1000 [00:00<?, ?it/s]

Early quitting at epoch: 472


  0%|          | 1/1000 [00:00<02:51,  5.83it/s]

...

Early quitting at epoch: 32


  0%|          | 2/1000 [00:00<01:09, 14.42it/s]

Early quitting at epoch: 23


  2%|▏         | 22/1000 [00:01<01:11, 13.68it/s]

Early quitting at epoch: 22


In [10]:
# Save all trained models
dir = f"{models_dir}/{dataset_method}"
trainer.save_trained_models(dir=dir, to_be_saved_models=None)